In [1]:
!pip install transformers

In [2]:
!pip install pymysql

In [3]:
!pip install faiss-cpu

In [4]:
!pip install sentence-transformers

In [5]:
import pandas as pd
import numpy as np
import faiss
import pymysql.cursors
from sentence_transformers import SentenceTransformer
from transformers import BartForConditionalGeneration, BartTokenizer
import torch

C:\Users\goutam.tak\Anaconda\Lib\site-packages\sentence_transformers\cross_encoder\CrossEncoder.py:11: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange


In [97]:
# Load the CSV file
df = pd.read_csv(r'C:\Users\goutam.tak\Downloads\employee_queries_dataset_large.csv')


In [98]:
# Clean the 'Prompt' column to remove NaNs and non-string values
df['Prompt'] = df['Prompt'].fillna('').astype(str)

In [9]:
# Load Sentence Transformer model
model = SentenceTransformer('all-MiniLM-L6-v2')

C:\Users\goutam.tak\Anaconda\Lib\site-packages\huggingface_hub\file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [99]:
# Generate embeddings for the 'Prompt' column
df['embeddings'] = df['Prompt'].apply(lambda x: model.encode(x))

In [100]:
# Convert embeddings to a format suitable for FAISS
embeddings = np.stack(df['embeddings'].values.tolist())

In [101]:
# Initialize FAISS index
d = embeddings.shape[1]
index = faiss.IndexFlatL2(d)
index.add(embeddings)

In [102]:
# Function to encode a query using Sentence Transformer
def encode_query(query, model):
    return model.encode(query)
def semantic_search_top_query(query, model, index, df, threshold=0.8):
    query_embedding = encode_query(query, model)
    D, I = index.search(np.array([query_embedding]), k=1)
    distance = D[0][0]
    retrieved_query = df.iloc[I[0][0]]['Query']

    # Check if the distance is below the threshold
    if distance <= threshold:
        return retrieved_query
    else:
        return None

In [14]:
!pip install mysql-connector-python

   ---------------------------------------- 0.0/14.5 MB ? eta -:--:--
   ---------------------------------------- 0.0/14.5 MB ? eta -:--:--
   ---------------------------------------- 0.0/14.5 MB 165.2 kB/s eta 0:01:28
   ---------------------------------------- 0.0/14.5 MB 163.8 kB/s eta 0:01:29
   ---------------------------------------- 0.0/14.5 MB 178.6 kB/s eta 0:01:21
   ---------------------------------------- 0.1/14.5 MB 327.3 kB/s eta 0:00:44
   ---------------------------------------- 0.2/14.5 MB 614.4 kB/s eta 0:00:24
    --------------------------------------- 0.3/14.5 MB 896.4 kB/s eta 0:00:16
   - -------------------------------------- 0.4/14.5 MB 1.2 MB/s eta 0:00:13
   -- ------------------------------------- 0.8/14.5 MB 2.0 MB/s eta 0:00:07
   ---- ----------------------------------- 1.5/14.5 MB 3.6 MB/s eta 0:00:04
   ------- -------------------------------- 2.6/14.5 MB 5.6 MB/s eta 0:00:03
   ----------- ---------------------------- 4.2/14.5 MB 8.0 MB/s eta 0:00:02
 

In [15]:
!pip install mysql-connector-python

In [113]:
# Example usage: Semantic search and execute SQL query
query = "Retrieve the full names and project names of employees who worked on projects"
retrieved_query = semantic_search_top_query(query, model, index, df)

In [114]:
retrieved_query

'SELECT e.full_name, p.project_name FROM employees e JOIN employee_projects p ON e.employee_id = p.employee_id;'

In [115]:
# Placeholder function to execute an SQL query (adjust according to your database)
import mysql.connector

def execute_sql_query(query):
    try:
        # Connect to the MySQL database
        connection = mysql.connector.connect(
           host='ATMECSINLT-545',  # Adjust host if necessary
            user='root',  # Replace with your MySQL username
            password='1234',  # Replace with your MySQL password
            database='task1'  # Database name
        )
        cursor = connection.cursor()
        cursor.execute(query)
        results = cursor.fetchall()
        connection.close()
        return results
    except Exception as e:
        print(f"An error occurred while executing the SQL query: {e}")
        return None

In [117]:
def generate_response_with_bart(results):
    input_text = " ".join([str(row) for row in results])
    input_ids = BartTokenizer.from_pretrained('facebook/bart-large').encode(input_text, return_tensors='pt')

    # Load BART model and tokenizer
    bart_model = BartForConditionalGeneration.from_pretrained('facebook/bart-large')
    
    with torch.no_grad():
        output_ids = bart_model.generate(input_ids, max_length=512, num_beams=5, early_stopping=True)

    response = BartTokenizer.from_pretrained('facebook/bart-large').decode(output_ids[0], skip_special_tokens=True)
    return response

In [118]:
def process_query(user_query, model, index, df):
    retrieved_query = semantic_search_top_query(user_query, model, index, df)
    if retrieved_query:
        print(f"Retrieved SQL Query: {retrieved_query}")

        results = execute_sql_query(retrieved_query)
        if results:
            print("\nSQL Query Results:")
            for row in results:
                print(f"Result: {row}")  # Adjust printing format based on your result structure

            response = generate_response_with_bart(results)
            print("\nGenerated Response:")
            print(response)
        else:
            print("No results found for the executed query.")
    else:
        print("No relevant query found.")


In [122]:
query=input("")
process_query(query, model, index, df)

 Retrieve the full names and project names of employees who worked on projects


Retrieved SQL Query: SELECT e.full_name, p.project_name FROM employees e JOIN employee_projects p ON e.employee_id = p.employee_id;

SQL Query Results:
Result: ('John Doe', 'Project Alpha')
Result: ('Jane Smith', 'Project Beta')
Result: ('Emily Johnson', 'Project Gamma')

Generated Response:
('John Doe', 'Project Alpha') ('John Doe, 'Project Beta') ('Emily Johnson', 'project Gamma')


In [124]:
import pickle

In [126]:
import pickle
import faiss

# Save SentenceTransformer model
model.save('sentence_transformer_model')

# Save FAISS index
faiss.write_index(index, 'faiss_index')

# Save DataFrame and other necessary objects
with open('data_and_functions.pkl', 'wb') as f:
    pickle.dump({'df': df, 'encode_query': encode_query, 'semantic_search_top_query': semantic_search_top_query, 'execute_sql_query': execute_sql_query, 'generate_response_with_bart': generate_response_with_bart, 'process_query': process_query}, f)


In [127]:
import pickle
import faiss
from sentence_transformers import SentenceTransformer

# Load SentenceTransformer model
model = SentenceTransformer('sentence_transformer_model')

# Load FAISS index
index = faiss.read_index('faiss_index')

# Load DataFrame and other necessary objects
with open('data_and_functions.pkl', 'rb') as f:
    data_and_functions = pickle.load(f)

df = data_and_functions['df']
encode_query = data_and_functions['encode_query']
semantic_search_top_query = data_and_functions['semantic_search_top_query']
execute_sql_query = data_and_functions['execute_sql_query']
generate_response_with_bart = data_and_functions['generate_response_with_bart']
process_query = data_and_functions['process_query']


In [132]:
# Save model, index, and data
model.save('sentence_transformer_model')
faiss.write_index(index, 'faiss_index.index')
with open('data_and_functions.pkl', 'wb') as f:
    pickle.dump({'df': df, 'encode_query': encode_query, 'semantic_search_top_query': semantic_search_top_query, 'execute_sql_query': execute_sql_query, 'generate_response_with_bart': generate_response_with_bart, 'process_query': process_query}, f)


In [134]:
import pickle
from IPython.display import FileLink, display
import shutil
import os

In [131]:
from IPython.display import FileLink, display

# Create download links for individual files
display(FileLink('sentence_transformer_model.zip'))
display(FileLink('faiss_index.index'))
display(FileLink('data_and_functions.pkl'))



C:\Users\goutam.tak\sentence_transformer_model.zip

C:\Users\goutam.tak\faiss_index.index

C:\Users\goutam.tak\data_and_functions.pkl

In [135]:
# Compress the SentenceTransformer model directory into a zip file
shutil.make_archive('sentence_transformer_model', 'zip', 'sentence_transformer_model')


'C:\\Users\\goutam.tak\\sentence_transformer_model.zip'

In [136]:
# Create download links
print("Download links for the saved files:")
display(FileLink('sentence_transformer_model.zip'))
display(FileLink('faiss_index.index'))
display(FileLink('data_and_functions.pkl'))

Download links for the saved files:


C:\Users\goutam.tak\sentence_transformer_model.zip

C:\Users\goutam.tak\faiss_index.index

C:\Users\goutam.tak\data_and_functions.pkl